# Importing llamas_pyjamas modules

In [ ]:
import os
import sys
import numpy as np
import pickle
import ray
import pkg_resources
import glob
import traceback
from pathlib import Path
from config import BASE_DIR, OUTPUT_DIR, DATA_DIR


# Get package root and add to path before other imports
package_root = Path().absolute().parent
sys.path.append(str(package_root))
sys.path.append(BASE_DIR+'/')


ray.init(ignore_reinit_error=True)
from llamas_pyjamas.Trace.traceLlamasMulti import main # type: ignore
from llamas_pyjamas.Extract.extractLlamas import ExtractLlamas, save_extractions, load_extractions # type: ignore
from llamas_pyjamas.Image.WhiteLight import WhiteLight, WhiteLightFits
from llamas_pyjamas.QA.llamasQA import plot_ds9



In [ ]:
# Get absolute path to llamas_pyjamas package
package_path = pkg_resources.resource_filename('llamas_pyjamas', '')
package_root = os.path.dirname(package_path)

print(f"Package path: {package_path}")
print(f"Package root: {package_root}")

In [ ]:
# Configure Ray runtime environment
runtime_env = {
    "py_modules": [package_root],
    "env_vars": {"PYTHONPATH": f"{package_root}:{os.environ.get('PYTHONPATH', '')}"},
    "excludes": [
        str(Path(DATA_DIR) / "**"),  # Exclude DATA_DIR and all subdirectories
        "**/*.fits",                 # Exclude all FITS files anywhere
        "**/*.pkl",                  # Exclude all pickle files anywhere
        "**/.git/**",               # Exclude git directory
    ]
}

# Initialize Ray
ray.shutdown()
ray.init(runtime_env=runtime_env)

Running the trace finding algorithm on all of the HDU extensions

In [ ]:
#fitsfile = '/Users/slh/Documents/Projects/Magellan_dev/LLAMAS/LLAMAS_2024-11-20T13_15_33.000_mef.fits'
filename = os.path.join(DATA_DIR, 'LLAMAS_2024-08-23T16_12_14.742_mef.fits')
main(filename)

In [ ]:

def extract_traces(files):
    extraction_list = []
    for file in files:
        try:
            with open(file, mode='rb') as f:
                tracer = pickle.load(f)
            extraction = ExtractLlamas(tracer)
            extraction_list.append(extraction)
        except Exception as e:
            print(f"Error extracting trace from {file}")
            print(traceback.format_exc())
    filename = save_extractions(extraction_list)
    return filename

In [ ]:
os.listdir(OUTPUT_DIR)
files = glob.glob(os.path.join(OUTPUT_DIR,'*traces.pkl'))
extracted_object_file = extract_traces(files)

In [ ]:
print(extracted_object_file)
obj, metadata = load_extractions(os.path.join(OUTPUT_DIR, extracted_object_file))
WhiteLightFits(obj)